In [1]:
%pip install --quiet -U azureml-fsspec dotenv
%cd Users/djenk.ivanov/


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
/mnt/batch/tasks/shared/LS_root/mounts/clusters/movielens-compute/code/Users/djenk.ivanov


In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from dotenv import load_dotenv
import os

load_dotenv()

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=os.getenv("AZURE_SUBSCRIPTION_ID"),
    resource_group_name=os.getenv("AZURE_RESOURCE_GROUP"),
    workspace_name=os.getenv("AZURE_ML_WORKSPACE"),
)

In [3]:
import pandas as pd

data_asset = ml_client.data.get(name="movielens32m", version="1")
print(f'Data asset path: {data_asset.path}')

ratings = pd.read_csv(data_asset.path + "/ratings.csv")
movies = pd.read_csv(data_asset.path + "/movies.csv")

Data asset path: azureml://subscriptions/ba117730-305b-41b8-8447-bf6834f0a56f/resourcegroups/djenk-rg1/workspaces/djenk-ml/datastores/movielens32m_ds/paths/ml-32m/


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()


In [4]:
ratings.head() 

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
hot_encode_genres = pd.Series(movies['genres']).str.get_dummies(sep='|')
encoded_movies = movies.merge(hot_encode_genres, left_index=True, right_index=True)
encoded_movies.drop(columns=['genres'], inplace=True)
encoded_movies.head()

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
ratings = ratings.drop(columns=['timestamp'])
merged = ratings.merge(encoded_movies, on='movieId', how='left')

In [8]:
print(ratings.shape, movies.shape, merged.shape)
merged.head()

(32000204, 3) (87585, 3) (32000204, 24)


,userId,movieId,rating,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,17,4.0,Sense and Sensibility (1995),0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,25,1.0,Leaving Las Vegas (1995),0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,29,2.0,"City of Lost Children, The (Cité des enfants p...",0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,1,30,5.0,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,32,5.0,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,0,0


In [9]:
import numpy as np

df = merged.copy()

df['liked'] = (df['rating'] >= 3).astype('int8')

row_hash = pd.util.hash_pandas_object(df[['userId', 'movieId', 'liked']], index=False).astype('uint64')
rand = (row_hash % (2**32)) / (2**32)

df['split'] = np.where(rand < 0.5, 'train', 'test')

In [10]:
train = df[df['split'] == 'train']
test = df[df['split'] == 'test']

In [11]:
train.head()

,userId,movieId,rating,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,liked,split
2,1,29,2.0,"City of Lost Children, The (Cité des enfants p...",0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,train
5,1,34,2.0,Babe (1995),0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,train
6,1,36,1.0,Dead Man Walking (1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,train
10,1,161,1.0,Crimson Tide (1995),0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,train
11,1,166,5.0,"Doom Generation, The (1995)",0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,train


In [12]:
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

user_encoder = OrdinalEncoder()
movie_encoder = OrdinalEncoder()

df['userId_enc'] = user_encoder.fit_transform(df[['userId']]).astype(np.int64)
df['movieId_enc'] = movie_encoder.fit_transform(df[['movieId']]).astype(np.int64)

X = csr_matrix((df['rating'].to_numpy(), (df['userId_enc'].to_numpy(), df['movieId_enc'].to_numpy())))

In [13]:
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 32000204 stored elements and shape (200948, 84432)>

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
V = cosine_similarity(X.T, X.T, dense_output=False)
V

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1765933664 stored elements and shape (84432, 84432)>

In [15]:
movies_by_id = movies.set_index("movieId")

def get_recommendations(title: str, top_n: int = 10):
    match = movies.loc[movies["title"] == title, "movieId"].values
    movie_id = int(match[0])

    movie_id_enc = int(movie_encoder.transform(pd.DataFrame([[movie_id]], columns=["movieId"]))[0, 0])

    row = V[movie_id_enc]
    sims = row.toarray().ravel()

    sims[movie_id_enc] = -np.inf

    idx = np.argpartition(sims, -top_n)[-top_n:]
    idx = idx[np.argsort(-sims[idx])]

    rec_movie_ids = (movie_encoder.inverse_transform(idx.reshape(-1, 1)).ravel().astype(int))
    
    return movies_by_id.loc[rec_movie_ids]


In [16]:
recommended = get_recommendations('Unfriended: Dark Web (2018)', top_n=10)
recommended

,title,genres
movieId,,
130636,Unfriended (2014),Horror|Mystery|Thriller
197343,Escape Room (2019),Action|Horror|Thriller
207890,Countdown (2019),Horror|Thriller
185989,Truth or Dare (2018),Horror|Thriller
194722,Look Away (2018),Horror|Thriller
197663,Happy Death Day 2U (2019),Horror|Mystery|Thriller
175649,Wish Upon (2017),Fantasy|Horror|Thriller
221396,Host (2020),Horror
168288,The Belko Experiment (2017),Action|Horror|Thriller


In [17]:
train = df[df['split'] == 'train'].copy()
test = df[df['split'] == 'test'].copy()

user_encoder = OrdinalEncoder()
movie_encoder = OrdinalEncoder()

train['userId_enc'] = user_encoder.fit_transform(train[['userId']]).astype(np.int64)
train['movieId_enc'] = movie_encoder.fit_transform(train[['movieId']]).astype(np.int64)



In [19]:
known_users = train['userId'].unique()
known_movies = train['movieId'].unique()

test = test[test['userId'].isin(known_users) & test['movieId'].isin(known_movies)].copy()

test['userId_enc'] = user_encoder.transform(test[['userId']]).astype(np.int64)
test['movieId_enc'] = movie_encoder.transform(test[['movieId']]).astype(np.int64)

In [ ]:

train_liked = train[train['liked'] == 1]
total_users = train['userId_enc'].max() + 1 
total_movies = train['movieId_enc'].max() + 1

X = csr_matrix((df['rating'].to_numpy(), (df['userId_enc'].to_numpy(), df['movieId_enc'].to_numpy())))

X_train = csr_matrix(   
    (np.ones(train_liked.shape[0]), (train_liked['userId_enc'].to_numpy(), train_liked['movieId_enc'].to_numpy())), shape=(total_users, total_movies)
)

V_train = cosine_similarity(X_train.T, X_train.T, dense_output=False)